In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


# Check if CUDA is installed
import torch

print(torch.cuda.is_available())




Tue Oct 31 11:33:56 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A4000                On | 00000000:00:0D.0 Off |                  Off |
| 41%   34C    P8               14W / 140W|      1MiB / 16376MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Install Dependencies

In [2]:
!pip install transformers
!pip install accelerate
!pip install peft
!pip install datasets
!pip install wandb
!pip install bitsandbytes
!pip install huggingface-hub
!pip install einops
!pip install pandas
!pip install numpy
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 72.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 64.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip
    

# Add credentials

In [ ]:
#!wandb login --relogin

In [3]:
from huggingface_hub import notebook_login

notebook_login()

# Download base model and dataset

In [1]:
from datasets import load_dataset
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
import torch

from transformers import BitsAndBytesConfig

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
)


quantization_config = BitsAndBytesConfig(
    llm_int8_threshold=10.,
)


model_name = "bigcode/starcoderbase-1b"
dataset_name = "flytech/llama-python-codes-30k"
dataset_split = "train[:5000]"

model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

dataset = load_dataset(dataset_name,split=dataset_split)


In [2]:
print(model.get_memory_footprint())

# 4bit is less than half of Jeton 2GB! 812699648
# 8bit = 1322307584 (1.3GB, almost okay)

1322307584


In [3]:

import sys
print("Python version:", sys.version)

Python version: 3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0]


# Prepare the dataset

In [4]:
def remove_llama_tokens(example):
  remove_tokens = ['[INST]','[/INST]','<s>','</s>', '```python', '```']
  for token in remove_tokens:
    example['instruction'] = example['instruction'].replace(token,'').strip()
    example['output'] = example['output'].replace(token,'').strip()
  example['text'] = '# '+example['instruction'] + '\n\n' + example['output']
  return example

def convert_to_pt(example):
  example['input_ids'] = torch.tensor(example['input_ids'])
  example['attention_mask'] = torch.tensor(example['attention_mask'])
  return example



def tokenize_function(examples, text_column='text'):
    return tokenizer(examples[text_column], return_tensors="pt",padding=True)


def prepare_datasets(tokenizer, dataset, test_size=.2):
  dataset = dataset.map(remove_llama_tokens)
  print(f"Tokenizing dataset ...")
  tokenized_dataset = dataset.map(tokenize_function, batched=True)



  return tokenized_dataset

In [5]:
dataset = dataset.map(remove_llama_tokens)


In [6]:
max_length = max([len(tokenizer(x['text'])["input_ids"]) for x in dataset])
print(max_length)

423


In [7]:

def preprocess_function(examples, text_column='text'):
    batch_size = len(examples[text_column])
    inputs = [str(x) for x in examples[text_column]]
    targets = [str(x) for x in examples[text_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets, add_special_tokens=False)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset.column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [8]:
processed_datasets = processed_datasets.train_test_split(test_size=0.1)

In [9]:
model

GPTBigCodeForCausalLM(
  (transformer): GPTBigCodeModel(
    (wte): Embedding(49152, 2048)
    (wpe): Embedding(8192, 2048)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPTBigCodeBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPTBigCodeAttention(
          (c_attn): Linear8bitLt(in_features=2048, out_features=2304, bias=True)
          (c_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTBigCodeMLP(
          (c_fc): Linear8bitLt(in_features=2048, out_features=8192, bias=True)
          (c_proj): Linear8bitLt(in_features=8192, out_features=2048, bias=True)
          (act): PytorchGELUTanh()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): L

# Load Model

In [10]:
from peft import LoraConfig, get_peft_model


peft_config = LoraConfig(
  lora_alpha=32,
  lora_dropout=.1,
  r=4,
  bias="none",
  task_type="CAUSAL_LM",
  target_modules="c_attn,c_proj,lm_head".split(","),
)



model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,998,848 || all params: 1,139,206,144 || trainable%: 0.17545972785764752


In [11]:
print(model.get_memory_footprint())
# just a biiiit larger!
# 1330302976 for 8bit with config above

1330302976


# Prepare Training!

In [12]:
from tqdm import tqdm
device = "cuda"
num_epochs = 5
lr=5e-5
batch_size = 64
iter_batch_size=2

log_interval = batch_size // iter_batch_size

In [13]:
train_dataloader = DataLoader(
    processed_datasets['train'], shuffle=True, collate_fn=default_data_collator, batch_size=iter_batch_size, pin_memory=True
)

eval_dataloader = DataLoader(
    processed_datasets['test'], shuffle=True, collate_fn=default_data_collator, batch_size=iter_batch_size, pin_memory=True
)

In [14]:


# model
# optimizer and lr scheduler
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

# training and evaluation
model = model.to(device);
for param in model.parameters():
    # Check if parameter dtype is  Half (float16)
    if param.dtype == torch.float16: 
        param.data = param.data.to(torch.float32)


In [15]:
def vibe_check_model(text = '# create function to print primes between 1 and 20'):

    vibe_check_input = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        inputs = {k: v.to(device) for k, v in vibe_check_input.items()}
        outputs = model.generate(
            input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=64 ,eos_token_id=0, temperature=0
        )

        return tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]


In [17]:
print(vibe_check_model('# check if int is prime or not'))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


# check if int is prime or not
def is_prime(n):
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    for i in range(3, int(n ** 0.5) + 1, 2):
        if n % i == 0:


In [18]:
# 8bit quant goes from 32 to 16!
print(model.get_memory_footprint())


1566285824


In [ ]:

# Next steps:

best_loss = 1e9

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        if step % log_interval == 0:
          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
            

    model.eval()
    eval_loss = 0
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()

    eval_epoch_loss = eval_loss / len(eval_dataloader)

    eval_epoch_loss = eval_epoch_loss.cpu().item()

    if eval_epoch_loss < best_loss:
      print('saving model to hub...')
      print("Showing the vibes...")
      print('===============================================')                   
      model_vibes = vibe_check_model()
      print(f"{model_vibes=}:")
      print('===============================================')           
      best_loss = eval_epoch_loss
      model.push_to_hub(repo_id='starcoder-python-1b')
      # save best model to HF Hub

    train_epoch_loss = total_loss / len(train_dataloader)
    train_epoch_loss = train_epoch_loss.cpu().item()

    print(f"{epoch=}: {train_epoch_loss=}  {eval_epoch_loss=}")

100%|██████████| 250/250 [00:42<00:00,  5.90it/s]
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


saving model to hub...
Showing the vibes...
model_vibes='# create function to print primes between 1 and 20\ndef print_primes(n):\n    for i in range(2, n):\n        if n % i == 0:\n            print(i)\n            break\n\n# create function to print primes between 1 and 20\ndef print_primes(n):\n    for i in':


adapter_model.bin:   0%|          | 0.00/8.05M [00:00<?, ?B/s]

epoch=0: train_epoch_loss=0.3001725971698761  eval_epoch_loss=0.20314191281795502


100%|██████████| 250/250 [00:42<00:00,  5.90it/s]
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


saving model to hub...
Showing the vibes...


In [64]:
vibe_check_input = tokenizer("# plot bar graph of pandas df", return_tensors="pt")
with torch.no_grad():
    inputs = {k: v.to(device) for k, v in vibe_check_input.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=256, eos_token_id=0    )
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


tensor([[   21,  6867,  5697,  3996,   432, 12643,  5018,   203,   203,    21,
           474,   203,    21,   904, 24411,  4447,   203,   203,   465, 12500,
            32, 17449,   619,  5743,   203,   465, 12643,   619,  6121,   203,
           203,  1479,   280,  6121,    32, 11364, 14178,    83,   950,   428,
            35,    30,   225,    36,    30,   225,    37,   614,   330,    84,
           950,   428,    38,    30,   225,    39,    30,   225,    40, 33993,
           203,  1479,   203,    21,   429,   203,   203,  1479,    32,  3399,
           346,   203,   203,    21,   474,   203,    21,  6867,  5697,  3996,
           432, 12643,  5018,   203,   203,    21,   904, 24411,  4447,   203,
           203,   465, 12500,    32, 17449,   619,  5743,   203,   465, 12643,
           619,  6121,   203,   203,  1479,   280,  6121,    32, 11364, 14178,
            83,   950,   428,    35,    30,   225,    36,    30,   225,    37,
           614,   330,    84,   950,   428,    38,  

# Load this model and adapter in the future

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token



peft_model_id = "nkasmanoff/starcoder-python-1b"
model_name = "bigcode/starcoderbase-1b"
model = AutoModelForCausalLM.from_pretrained(model_name)
model.load_adapter(peft_model_id)

Loading adapter weights from nkasmanoff/starcoder-python-1b led to unexpected keys not found in the model:  ['base_model.model.transformer.h.0.attn.c_attn.lora_A.default.weight', 'base_model.model.transformer.h.0.attn.c_attn.lora_B.default.weight', 'base_model.model.transformer.h.0.attn.c_proj.lora_A.default.weight', 'base_model.model.transformer.h.0.attn.c_proj.lora_B.default.weight', 'base_model.model.transformer.h.0.mlp.c_proj.lora_A.default.weight', 'base_model.model.transformer.h.0.mlp.c_proj.lora_B.default.weight', 'base_model.model.transformer.h.1.attn.c_attn.lora_A.default.weight', 'base_model.model.transformer.h.1.attn.c_attn.lora_B.default.weight', 'base_model.model.transformer.h.1.attn.c_proj.lora_A.default.weight', 'base_model.model.transformer.h.1.attn.c_proj.lora_B.default.weight', 'base_model.model.transformer.h.1.mlp.c_proj.lora_A.default.weight', 'base_model.model.transformer.h.1.mlp.c_proj.lora_B.default.weight', 'base_model.model.transformer.h.2.attn.c_attn.lora_A.de

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token


In [5]:
# training and evaluation
device = 'cuda'
model = model.to(device);


In [22]:
def vibe_check_model(text = '# create function to print primes between 1 and 20'):

    vibe_check_input = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        inputs = {k: v.to(device) for k, v in vibe_check_input.items()}
        outputs = model.generate(
            input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=64 ,eos_token_id=0, temperature=0
        )

        return tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]


In [23]:
print(vibe_check_model('# check if int is prime or not'))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


# check if int is prime or not
def is_prime(n):
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    for i in range(3, int(n**0.5)+1, 2):
        if n % i == 0:
            return False
